THIẾT LẬP VÀ CÀI ĐẶT THƯ VIỆN

In [5]:
# ===============================
# 🔧 Import Thư Viện
# ===============================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

import spacy
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from collections import Counter
import nltk
from nltk.translate.bleu_score import corpus_bleu
import os

# ===============================
# ⚙️ Thiết lập ban đầu
# ===============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

# ===============================
# ⚙️ Hằng số & Token đặc biệt
# ===============================
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
special_tokens = ['<unk>', '<pad>', '<sos>', '<eos>']
MAX_VOCAB_SIZE = 10000
BATCH_SIZE = 32
MAX_SENTENCE_LEN = 50
DATA_DIR = 'data/'

# ===============================
# 🧠 Tải Tokenizer spaCy thay thế torchtext
# ===============================
# Nếu chưa tải, mở terminal và chạy:
# python -m spacy download en_core_web_sm
# python -m spacy download fr_core_news_sm
spacy_en = spacy.load("en_core_web_sm")
spacy_fr = spacy.load("fr_core_news_sm")

def tokenize_en(text):
    """Tokenizer tiếng Anh"""
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

def tokenize_fr(text):
    """Tokenizer tiếng Pháp"""
    return [tok.text.lower() for tok in spacy_fr.tokenizer(text)]

# ===============================
# 🧩 Tạo Vocabulary (thay build_vocab_from_iterator)
# ===============================
def build_vocab(sentences, tokenizer, max_size=MAX_VOCAB_SIZE):
    counter = Counter()
    for sentence in sentences:
        counter.update(tokenizer(sentence))

    # Giới hạn kích thước vocab
    most_common = counter.most_common(max_size - len(special_tokens))
    vocab = {word: idx + len(special_tokens) for idx, (word, _) in enumerate(most_common)}
    # Thêm token đặc biệt
    for idx, token in enumerate(special_tokens):
        vocab[token] = idx
    return vocab

# ===============================
# 🔁 Hàm chuyển văn bản sang ID
# ===============================
def encode_sentence(sentence, vocab, tokenizer, max_len=MAX_SENTENCE_LEN):
    tokens = tokenizer(sentence)[:max_len-2]
    ids = [vocab.get('<sos>')] + [vocab.get(tok, vocab['<unk>']) for tok in tokens] + [vocab.get('<eos>')]
    return torch.tensor(ids, dtype=torch.long)

# ===============================
# 📦 Dataset tùy chỉnh (thay torchtext Dataset)
# ===============================
class TranslationDataset(Dataset):
    def __init__(self, src_texts, trg_texts, src_vocab, trg_vocab, src_tokenizer, trg_tokenizer):
        self.src_texts = src_texts
        self.trg_texts = trg_texts
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer

    def __len__(self):
        return len(self.src_texts)

    def __getitem__(self, idx):
        src_tensor = encode_sentence(self.src_texts[idx], self.src_vocab, self.src_tokenizer)
        trg_tensor = encode_sentence(self.trg_texts[idx], self.trg_vocab, self.trg_tokenizer)
        return src_tensor, trg_tensor

# ===============================
# 🧮 Collate function để padding batch
# ===============================
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_padded = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    trg_padded = pad_sequence(trg_batch, padding_value=PAD_IDX, batch_first=True)
    return src_padded, trg_padded


Sử dụng thiết bị: cpu


XỬ LÝ DỮ LIỆU CƠ BẢN

In [8]:
import os
import spacy

# ===============================
# 🧠 Tokenizers (thay cho torchtext)
# ===============================
# Nếu chưa tải, mở terminal và chạy:
# python -m spacy download en_core_web_sm
# python -m spacy download fr_core_news_sm

# Tải mô hình spaCy
spacy_en = spacy.load("en_core_web_sm")
spacy_fr = spacy.load("fr_core_news_sm")

# Hàm token hóa thủ công
def en_tokenizer(text):
    """Tokenizer tiếng Anh"""
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

def fr_tokenizer(text):
    """Tokenizer tiếng Pháp"""
    return [tok.text.lower() for tok in spacy_fr.tokenizer(text)]


# ===============================
# 📂 Hàm đọc dữ liệu Multi30K
# ===============================
def read_pairs(file_en, file_fr, data_dir='data/', max_len=50):
    """
    Đọc dữ liệu từ tệp song ngữ (Anh - Pháp),
    lọc câu quá dài, và trả về danh sách (en, fr).
    """
    path_en = os.path.join(data_dir, file_en)
    path_fr = os.path.join(data_dir, file_fr)
    data = []

    try:
        with open(path_en, 'r', encoding='utf-8') as f_en, open(path_fr, 'r', encoding='utf-8') as f_fr:
            for en_line, fr_line in zip(f_en, f_fr):
                en_clean = en_line.strip()
                fr_clean = fr_line.strip()

                # Bỏ câu trống hoặc quá dài
                if 0 < len(en_clean.split()) <= max_len and 0 < len(fr_clean.split()) <= max_len:
                    data.append((en_clean, fr_clean))

        print(f"✅ Đã tải {len(data)} cặp câu từ {file_en} & {file_fr}")
        return data

    except FileNotFoundError:
        print(f"❌ LỖI: Không tìm thấy tệp. Kiểm tra đường dẫn: {path_en} hoặc {path_fr}")
        return []


# ===============================
# 🧩 Tải dữ liệu thực tế
# ===============================
DATA_DIR = 'data/'
MAX_SENTENCE_LEN = 50

train_data = read_pairs('data/train.en', 'data/train.fr', data_dir=DATA_DIR, max_len=MAX_SENTENCE_LEN)
val_data   = read_pairs('data/val.en',   'data/val.fr',   data_dir=DATA_DIR, max_len=MAX_SENTENCE_LEN)
test_data  = read_pairs('data/test.en',  'data/test.fr',  data_dir=DATA_DIR, max_len=MAX_SENTENCE_LEN)

if not train_data:
    raise RuntimeError("❗Không thể tải dữ liệu đào tạo. Vui lòng kiểm tra thư mục 'data/' và các tệp Multi30K.")


❌ LỖI: Không tìm thấy tệp. Kiểm tra đường dẫn: data/data/train.en hoặc data/data/train.fr
❌ LỖI: Không tìm thấy tệp. Kiểm tra đường dẫn: data/data/val.en hoặc data/data/val.fr
❌ LỖI: Không tìm thấy tệp. Kiểm tra đường dẫn: data/data/test.en hoặc data/data/test.fr


RuntimeError: ❗Không thể tải dữ liệu đào tạo. Vui lòng kiểm tra thư mục 'data/' và các tệp Multi30K.

XÂY DỰNG TỪ VỰNG VÀ DATALOADER

In [ ]:
# --- Xây dựng Từ vựng ---
def yield_tokens(data_iter, tokenizer, src_idx=0):
    for pair in data_iter:
        yield tokenizer(pair[src_idx])

en_vocab = build_vocab_from_iterator(
    yield_tokens(train_data, en_tokenizer, 0),
    min_freq=1, specials=special_tokens, max_tokens=MAX_VOCAB_SIZE
)
en_vocab.set_default_index(UNK_IDX)

fr_vocab = build_vocab_from_iterator(
    yield_tokens(train_data, fr_tokenizer, 1),
    min_freq=1, specials=special_tokens, max_tokens=MAX_VOCAB_SIZE
)
fr_vocab.set_default_index(UNK_IDX)

print(f"Kích thước từ vựng EN: {len(en_vocab)}, FR: {len(fr_vocab)}")

# --- Chuyển đổi Chuỗi thành Index ---
def data_process(data):
    data_index = []
    for en_sentence, fr_sentence in data:
        en_tensor = torch.tensor(en_vocab(en_tokenizer(en_sentence)), dtype=torch.long)
        fr_tensor = torch.tensor([SOS_IDX] + fr_vocab(fr_tokenizer(fr_sentence)) + [EOS_IDX], dtype=torch.long)
        data_index.append((en_tensor, fr_tensor))
    return data_index

train_data_indices = data_process(train_data)
val_data_indices = data_process(val_data)
test_data_indices = data_process(test_data)

# --- Custom Dataset và Collate Function ---
class TranslationDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(batch):
    # Sắp xếp batch và Padding
    batch.sort(key=lambda x: len(x[0]), reverse=True)
    en_batch, fr_batch = zip(*batch)
    en_lengths = torch.tensor([len(seq) for seq in en_batch], dtype=torch.long)
    en_padded = pad_sequence(en_batch, padding_value=PAD_IDX, batch_first=True)
    fr_padded = pad_sequence(fr_batch, padding_value=PAD_IDX, batch_first=True)
    
    return en_padded.to(device), en_lengths.to(device), fr_padded.to(device)

train_iterator = DataLoader(TranslationDataset(train_data_indices), batch_size=BATCH_SIZE, collate_fn=collate_fn)
valid_iterator = DataLoader(TranslationDataset(val_data_indices), batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_iterator = DataLoader(TranslationDataset(test_data_indices), batch_size=1, collate_fn=collate_fn)

ĐỊNH NGHĨA MÔ HÌNH

In [ ]:
class Encoder(nn.Module):
    # ... (code Encoder như trong utils.py)
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=PAD_IDX)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
    def forward(self, src, src_len):
        embedded = self.dropout(self.embedding(src))
        packed_embedded = pack_padded_sequence(embedded, src_len.cpu(), batch_first=True, enforce_sorted=True)
        _, (hidden, cell) = self.rnn(packed_embedded)
        return hidden, cell

class Decoder(nn.Module):
    # ... (code Decoder như trong utils.py)
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=PAD_IDX)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1) 
        embedded = self.dropout(self.embedding(input)) 
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        output_1d = output.squeeze(1) 
        prediction = self.fc_out(output_1d)
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    # ... (code Seq2Seq như trong utils.py)
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hid_dim == decoder.hid_dim
        assert encoder.n_layers == decoder.n_layers
    def forward(self, src, src_len, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src, src_len)
        input = trg[:, 0]
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t] = output
            top1 = output.argmax(1) 
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[:, t] if teacher_force else top1
        return outputs

KHỞI TẠO MÔ HÌNH VÀ OPTIMIZER

In [ ]:
# Tham số mô hình
INPUT_DIM = len(en_vocab)
OUTPUT_DIM = len(fr_vocab)
EMB_DIM = 256 
HID_DIM = 512
N_LAYERS = 2
DROPOUT = 0.5
TRG_PAD_IDX = PAD_IDX

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m):
    # Khởi tạo trọng số
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)
print("Đã khởi tạo mô hình và trọng số.")

# Optimizer & Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)
CLIP = 1.0 # Gradient Clipping

ĐỊNH NGHĨA HÀM HUẤN LUYỆN VÀ ĐÁNH GIÁ

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    """Huấn luyện mô hình trong một epoch."""
    model.train()
    epoch_loss = 0
    for i, (src, src_len, trg) in enumerate(tqdm(iterator, desc="Training")):
        optimizer.zero_grad()
        output = model(src, src_len, trg, teacher_forcing_ratio=0.5)
        
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    """Đánh giá mô hình (Validation) với Teacher Forcing = 0."""
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, (src, src_len, trg) in enumerate(tqdm(iterator, desc="Evaluating")):
            output = model(src, src_len, trg, 0) # Teacher Forcing Ratio = 0
            
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

VÒNG LẶP HUẤN LUYỆN CHÍNH

In [ ]:
N_EPOCHS = 20
best_valid_loss = float('inf')
train_losses = []
valid_losses = []
patience = 0 

if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')

print("--- Bắt đầu Huấn luyện ---")
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'checkpoints/best_model.pth')
        patience = 0
        print(f'*** Epoch {epoch+1:02}: Lưu mô hình tốt nhất! ***')
    else:
        patience += 1
    
    print(f'Epoch: {epoch+1:02} | Patience: {patience}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid PPL: {np.exp(valid_loss):7.3f}')

    if patience >= 3:
        print(f"\n--- Dừng sớm tại Epoch {epoch+1} ---")
        break

# Tải mô hình tốt nhất để sử dụng cho Inference
model.load_state_dict(torch.load('checkpoints/best_model.pth'))
print("\nĐã tải mô hình tốt nhất.")

HÀM DỊCH (INFERENCE) VÀ ĐÁNH GIÁ BLEU

In [ ]:
def translate(sentence, en_vocab, fr_vocab, en_tokenizer, model, device, max_len=50):
    """Thực hiện dịch câu đơn bằng Greedy Decoding."""
    model.eval()
    
    tokens = en_tokenizer(sentence)
    src_indexes = [en_vocab[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device) 
    src_len = torch.LongTensor([len(src_indexes)]).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor, src_len)

    input = torch.LongTensor([SOS_IDX]).to(device) 
    trg_indexes = []

    for t in range(max_len):
        with torch.no_grad():
            output, hidden, cell = model.decoder(input, hidden, cell)
        
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        if pred_token == EOS_IDX:
            break
            
        input = torch.LongTensor([pred_token]).to(device)
        
    trg_tokens = [fr_vocab.lookup_token(i) for i in trg_indexes]
    
    translation = ' '.join(trg_tokens[1:-1])
    return translation

def calculate_bleu_score(test_data_indices, model, device):
    """Tính toán BLEU Score trên toàn bộ tập test."""
    trgs = []
    prds = []
    
    for src_tensor, trg_tensor in tqdm(test_data_indices, desc="Tính BLEU"):
        
        # Chuyển tensor thành chuỗi tiếng Anh để dùng hàm translate
        src_sentence = [en_vocab.lookup_token(i.item()) for i in src_tensor]
        sentence_str = ' '.join([t for t in src_sentence if t not in special_tokens])
        predicted_str = translate(sentence_str, en_vocab, fr_vocab, en_tokenizer, model, device)
        predicted_tokens = predicted_str.split()
        
        # Ground Truth
        target_tokens = [fr_vocab.lookup_token(i.item()) for i in trg_tensor]
        clean_target = [t for t in target_tokens if t not in special_tokens]
        
        trgs.append([clean_target]) 
        prds.append(predicted_tokens)

    bleu_score = corpus_bleu(trgs, prds)
    return bleu_score

KẾT QUẢ CUỐI CÙNG VÀ ĐỒ THỊ

In [ ]:
# Tính toán BLEU Score
print("\n--- 6.5. Đánh giá ---")
bleu_score = calculate_bleu_score(test_data_indices, model, device)
print(f"BLEU Score trên tập Test: {bleu_score*100:.2f}")

# Ví dụ Dịch
print("\n--- Ví dụ Dịch ---")
example_sentence = "A man is eating spaghetti."
translation_output = translate(example_sentence, en_vocab, fr_vocab, en_tokenizer, model, device)
print(f"Câu nguồn (EN): {example_sentence}")
print(f"Dịch (FR): {translation_output}")


# Đồ thị Loss
if train_losses and valid_losses:
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(valid_losses, label='Validation Loss')
    plt.title('Train vs. Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Lưu biểu đồ (Bắt buộc cho báo cáo)
    plt.savefig('checkpoints/train_val_loss.png') 
    plt.show()